<a href="https://colab.research.google.com/github/aaronjoseph/AB_Testing/blob/master/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
import os
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split

nltk.download('stopwords')
  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
imdb_data = pd.read_csv('IMDB Dataset.csv',error_bad_lines=False,engine='python')

Skipping line 31643: unexpected end of data


In [48]:
# Data Cleaning Process
def preprocess(text):
  text=text.lower()
  # remove hyperlinks
  text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
  text = re.sub(r'http?:\/\/.*[\r\n]*', '', text)
  #remove hashtag sign
  text=re.sub(r"#","",text)   
  #remove mentions
  text = re.sub(r"(?:\@)\w+", '', text)
  #remove non ascii chars
  text=text.encode("ascii",errors="ignore").decode()
  #remove some puncts (except . ! ?)
  text=re.sub(r'[:"#$%&\*+,-/:;<=>@\\^_`{|}~]+','',text)
  text=re.sub(r'[!]+','!',text)
  text=re.sub(r'[?]+','?',text)
  text=re.sub(r'[.]+','.',text)
  text=re.sub(r"'","",text)
  text=re.sub(r"\(","",text)
  text=re.sub(r"\)","",text)    
  text=" ".join(text.split())
  return text

def simple_stemmer(text):
  ps=nltk.porter.PorterStemmer()
  text= ' '.join([ps.stem(word) for word in text.split()])
  return text

#removing the stopwords
def remove_stopwords(text, is_lower_case=False):
  #set stopwords to english
  stop=set(stopwords.words('english'))
  # Tokenization
  tokenizer=ToktokTokenizer()
  #Setting English stopwords
  stopword_list=nltk.corpus.stopwords.words('english')
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
      filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
      filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text = ' '.join(filtered_tokens)    
  return filtered_text

In [49]:
#Apply function
imdb_data['clean']=imdb_data['review'].progress_apply(preprocess)
imdb_data['stem']=imdb_data['clean'].progress_apply(simple_stemmer)
imdb_data['token'] = imdb_data['stem'].progress_apply(remove_stopwords) 

100%|██████████| 31641/31641 [00:36<00:00, 859.50it/s]


# Bag of Words & TFIDF

In [55]:
# Splitting the data
X = imdb_data['token']
y = imdb_data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 

21941    thi one best film ive seen last yearsbelmonndo...
3207     star rate saturday night friday night friday m...
2319     film manag surviv almost origin alon wonderlan...
17226    took three attempt get thi movi although total...
31498    dure sleepless night wa switch channel found t...
                               ...                        
29802    wa first introduc thi movi san antonio tx thi ...
5390     cant knock thi film terribl becaus obviou midw...
860      thi product wa quit surpris absolut love obscu...
15795    thi decent movi although littl bit short time ...
23654    anoth veri good mann flick thank fatherson com...
Name: token, Length: 25312, dtype: object